<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [11]:
import os
from PIL import Image
from skimage import io

forestspath = './data/forest/'
forests = []
valid_files = ['.jpg']
for image in os.listdir(forestspath):
    ext = os.path.splitext(image)[1]
    if ext.lower() not in valid_files:
        continue
    forests.append(Image.open(os.path.join(forestspath,image)))

forests[:10]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9908>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD99B0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9A58>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9B00>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9BA8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9C50>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9D30>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9DD8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9E80>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9828>]

In [12]:
mountainspath = './data/mountain/'
mountains = []
valid_files = ['.jpg']
for image in os.listdir(mountainspath):
    ext = os.path.splitext(image)[1]
    if ext.lower() not in valid_files:
        continue
    mountains.append(Image.open(os.path.join(mountainspath,image)))

mountains[:10]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF150B8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BDD9780>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF151D0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF152B0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF15358>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF15400>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF154A8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF15550>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF155F8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x1FF9BF156A0>]

In [13]:
import numpy as np

y0 = np.zeros(len(forests))
y1 = np.ones(len(mountains))
y = np.hstack((y0, y1))

y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [18]:
X = []
for image in (forests + mountains):
    img_array = np.asarray(image)
    X.append(img_array)

print(len(X))

X = np.array(X)/255

print(X[0])

702
[[[0.01176471 0.         0.        ]
  [0.01960784 0.00392157 0.        ]
  [0.02352941 0.         0.        ]
  ...
  [0.         0.00392157 0.02352941]
  [0.03921569 0.03921569 0.07843137]
  [0.         0.         0.03529412]]

 [[0.01568627 0.         0.        ]
  [0.02745098 0.00784314 0.        ]
  [0.03137255 0.         0.        ]
  ...
  [0.04313725 0.04705882 0.06666667]
  [0.01176471 0.01568627 0.03529412]
  [0.         0.         0.03137255]]

 [[0.02745098 0.         0.        ]
  [0.03529412 0.00392157 0.        ]
  [0.03921569 0.         0.        ]
  ...
  [0.00392157 0.00392157 0.01176471]
  [0.         0.         0.00784314]
  [0.01568627 0.01568627 0.02352941]]

 ...

 [[0.00784314 0.         0.00392157]
  [0.05882353 0.04313725 0.04705882]
  [0.03137255 0.01568627 0.01960784]
  ...
  [0.05490196 0.01960784 0.        ]
  [0.03529412 0.         0.        ]
  [0.04705882 0.01176471 0.        ]]

 [[0.00784314 0.00392157 0.02352941]
  [0.01176471 0.00784314 0.027450

In [21]:
X[0].shape

(256, 256, 3)

In [22]:
y[0]

0.0

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Instatiate Model

In [49]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

In [71]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [72]:
# Compile Model

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/Users/jonathansokoll/anaconda3/envs/U4-S3-DNN/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


## Fit Model

In [73]:
model.fit(X_train, y_train, 
          epochs=20, 
          validation_data=(X_test, y_test)
         )

Train on 470 samples, validate on 232 samples
Epoch 1/20
470/470 [==============================] - 140s 297ms/sample - loss: 0.5711 - accuracy: 0.7787 - val_loss: 0.6928 - val_accuracy: 0.5043
Epoch 2/20
470/470 [==============================] - 135s 287ms/sample - loss: 0.3721 - accuracy: 0.8660 - val_loss: 1.6663 - val_accuracy: 0.4957
Epoch 3/20
470/470 [==============================] - 120s 255ms/sample - loss: 0.1016 - accuracy: 0.9809 - val_loss: 2.4377 - val_accuracy: 0.4957
Epoch 4/20
470/470 [==============================] - 120s 254ms/sample - loss: 0.1012 - accuracy: 0.9787 - val_loss: 0.6927 - val_accuracy: 0.4957
Epoch 5/20
470/470 [==============================] - 121s 257ms/sample - loss: 0.0645 - accuracy: 0.9766 - val_loss: 0.6821 - val_accuracy: 0.5647
Epoch 6/20
470/470 [==============================] - 120s 255ms/sample - loss: 0.0691 - accuracy: 0.9702 - val_loss: 2.2906 - val_accuracy: 0.4957
Epoch 7/20
470/470 [==============================] - 120s 255ms/s

### Well now, that is quite the poor performance.

Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 39s 69ms/sample - loss: 0.1216 - accuracy: 0.9715 - val_loss: 1.4221 - val_accuracy: 0.0000e+00
Epoch 2/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0410 - accuracy: 0.9875 - val_loss: 2.9222 - val_accuracy: 0.0000e+00
Epoch 3/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0646 - accuracy: 0.9697 - val_loss: 0.9962 - val_accuracy: 0.0000e+00
Epoch 4/5
561/561 [==============================] - 39s 70ms/sample - loss: 0.0723 - accuracy: 0.9733 - val_loss: 0.2627 - val_accuracy: 1.0000
Epoch 5/5
561/561 [==============================] - 38s 68ms/sample - loss: 0.0333 - accuracy: 0.9893 - val_loss: 0.4859 - val_accuracy: 1.0000


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [39]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [62]:
# Setup Architecture

model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(8, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(4, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 125, 125, 32)      18464     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 60, 60, 16)        4624      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 28, 28, 8)        

In [63]:
# Compile Model

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
# Fit Model
stop= EarlyStopping(monitor='val_accuracy', min_delta=.01, patience=3)

model.fit(X_train, y_train, 
          epochs=20, 
          validation_data=(X_test, y_test),
          callbacks=[stop]
         )

Train on 470 samples, validate on 232 samples
Epoch 1/20
470/470 [==============================] - 26s 56ms/sample - loss: 0.5109 - accuracy: 0.7596 - val_loss: 0.8200 - val_accuracy: 0.6853
Epoch 2/20
470/470 [==============================] - 26s 55ms/sample - loss: 0.3109 - accuracy: 0.8638 - val_loss: 0.2510 - val_accuracy: 0.8966
Epoch 3/20
470/470 [==============================] - 26s 55ms/sample - loss: 0.2594 - accuracy: 0.9000 - val_loss: 0.2075 - val_accuracy: 0.9138
Epoch 4/20
470/470 [==============================] - 26s 55ms/sample - loss: 0.1897 - accuracy: 0.9298 - val_loss: 0.1906 - val_accuracy: 0.9267
Epoch 5/20
470/470 [==============================] - 26s 56ms/sample - loss: 0.1750 - accuracy: 0.9362 - val_loss: 0.1760 - val_accuracy: 0.9310
Epoch 6/20
470/470 [==============================] - 26s 56ms/sample - loss: 0.1535 - accuracy: 0.9532 - val_loss: 0.1998 - val_accuracy: 0.9224
Epoch 7/20
470/470 [==============================] - 26s 56ms/sample - loss: 

### Looks like my validation accuracy ended up at just shy of 93%

In [65]:
# Fit Model
model.fit(X, y,
          #X_train, y_train, 
          epochs=20, 
          validation_split=0.2,
          shuffle=True,
          callbacks=[stop]
          #validation_data=(X_test, y_test))
         )

Train on 561 samples, validate on 141 samples
Epoch 1/20
561/561 [==============================] - 31s 55ms/sample - loss: 0.1789 - accuracy: 0.9358 - val_loss: 0.0607 - val_accuracy: 0.9858
Epoch 2/20
561/561 [==============================] - 30s 53ms/sample - loss: 0.1491 - accuracy: 0.9376 - val_loss: 0.9937 - val_accuracy: 0.6028
Epoch 3/20
561/561 [==============================] - 30s 53ms/sample - loss: 0.1969 - accuracy: 0.9234 - val_loss: 0.0945 - val_accuracy: 0.9787
Epoch 4/20
561/561 [==============================] - 30s 53ms/sample - loss: 0.1594 - accuracy: 0.9376 - val_loss: 0.3757 - val_accuracy: 0.8865


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 215, 215, 32)      9632      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 39, 39, 64)        51264     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 64)          102464    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [96]:
# Compile Model


In [99]:
# Fit Model


Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2667 - accuracy: 0.9073 - val_loss: 0.1186 - val_accuracy: 0.9858
Epoch 2/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2046 - accuracy: 0.9073 - val_loss: 0.3342 - val_accuracy: 0.8511
Epoch 3/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1778 - accuracy: 0.9287 - val_loss: 0.2746 - val_accuracy: 0.8723
Epoch 4/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1681 - accuracy: 0.9323 - val_loss: 0.8487 - val_accuracy: 0.5957
Epoch 5/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1606 - accuracy: 0.9394 - val_loss: 0.3903 - val_accuracy: 0.8582


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language